## [open in Kaggle](https://www.kaggle.com/meishidou/notebookc618242876)

# Prerequisites

In [ ]:
import json
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import transformers

# package in . directory
from bichoice import utils
from bichoice.data_processor import (
    C3BinaryExample,
    C3BinaryDataProcessor,
)

In [ ]:
assert transformers.__version__ == '4.1.1'

# declare a namespace
D = utils.GlobalSettings({
        'DATADIR': './data/',
        'MODELDIR': './outputs/bichoice_output/epoch-1/',
    })

# load training parameters
argD = utils.GlobalSettings.from_json(
    os.path.join(D.MODELDIR, 'global_settings.json'))
print('this model is trained with following hyper parameters:')
print(str(argD))

# Select an Example for Test

In [ ]:
def show_c3_example(e):
    '''show all info of a single example in C3'''
    print('-----PASSAGE-----')
    print('\n'.join(e.sentences))
    print('-----QUESTION-----')
    print(e.question)
    print('-----OPTIONS-----')
    for i, o in enumerate(e.options):
        print('    {}: {}'.format(chr(i+ord('A')), o))
    print('-----ANSWER-----')
    l = e.label
    print('    {}: {}'.format(chr(l+ord('A')), e.options[l]))

In [ ]:
test = utils.get_all_C3examples(os.path.join(D.DATADIR, 'test.json'))
test_e = random.choice(test)
show_c3_example(test_e)

# Reshape C3 as Binary Classification

In [ ]:
def C3Example_to_C3BinaryExample(eList):
    '''
    create `C3BinaryExample`s from `C3Example`s.
    
    Args
    ----
    `eList` : a list of `bichoice.utils.C3Example`
    
    Return
    ------
    `out` : a list of `bichioce.data_processor.C3BinaryExample`.
    '''
    out = []
    for e in eList:
        passage = ''.join(e.sentences)
        question = e.question
        answer = e.options[e.label]
        for o in e.options:
            if o == answer:
                continue
            out.append(C3BinaryExample(passage, question, answer, o, 0))
            out.append(C3BinaryExample(passage, question, o, answer, 1))
    return out

In [ ]:
def show_c3_binary_example(e):
    '''show all info of a single `C3BinaryExample`'''
    print('-----PASSAGE-----')
    print(e.passage)
    print('-----QUESTION-----')
    print(e.question)
    print('-----CHOICE_0-----')
    print(e.choice_0)
    print('-----CHOICE_1-----')
    print(e.choice_1)
    print('-----LABEL-----')
    print(e.label)

In [ ]:
# decomposite this example to several binary examples
test_b_e = C3Example_to_C3BinaryExample([test_e])
for i, e in enumerate(test_b_e):
    print('-----EXAMPLE{}-----'.format(i+1))
    show_c3_binary_example(e)

# Infer with Bi-Chioce Model

In [ ]:
# initialize model and load state dict from a checkpoint 
D.DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = transformers.BertTokenizer.from_pretrained(argD.MODEL_NAME)
processor = C3BinaryDataProcessor(tokenizer, argD.MAX_LENGTH)
model = transformers.BertForSequenceClassification.from_pretrained(
    argD.MODEL_NAME, num_labels=2)
model.load_state_dict(torch.load(os.path.join(D.MODELDIR, 'model.bin')))
model.to(D.DEVICE)
model.eval()

In [ ]:
def compare(D, model, processor, passage, question, c1, c2, avg=True):
    '''
    compare two options and select one with the higher probability.

    Args
    ----
    `D` : instance of `GlobalSettings`
    
    `model` : binary classifier

    `processor` : instance of `C3BinaryDataProcessor`

    `passage` : str

    `question` : str

    `c1` : str

    `c2` : str

    `avg` : bool, if `True`, the we switch the position of `c1` and `c2` 
            and average output as probability. 

    Return
    ------
    `label` : int, 0 means selecting c1, 1 means selecting c2
    '''
    # tokenizing
    example1 = C3BinaryExample(passage, question, c1, c2)
    example2 = C3BinaryExample(passage, question, c2, c1)
    f1 = processor.convert_example_to_features(example1)
    f2 = processor.convert_example_to_features(example2)
    batch = {
'input_ids': torch.LongTensor([f1.input_ids, f2.input_ids]),
'attention_mask': torch.LongTensor([f1.input_mask,f2.input_mask]),
'token_type_ids': torch.LongTensor([f1.segment_ids,f2.segment_ids]),
    }
    for k in batch:
        batch[k] = batch[k].to(D.DEVICE)

    model.to(D.DEVICE)
    model.eval()
    with torch.no_grad():
        output = model(**batch)
        logits = output.logits
    if avg:
        logit1 = (logits[0,0] + logits[1,1]).item()
        logit2 = (logits[0,1] + logits[1,0]).item()
    else:
        logit1 = logits[0,0].item()
        logit2 = logits[0,1].item()

    return int(logit2 > logit1)

In [ ]:
candidate = test_e.options[0]
# we sequentially compare all options
for c2 in test_e.options[1:]:
    print('we compare following options:')
    print('    option1:', candidate)
    print('    option2:', c2)
    bin_label = compare(D, model, processor, ''.join(test_e.sentences), 
                        test_e.question, candidate, c2)
    if bin_label == 1:
        print('and we select:', c2)
        candidate = c2
    else:
        print('and we select:', candidate)
print('----------')
print('infered answer:', candidate)
print('correct answer:', test_e.options[test_e.label])